# API attempt

The Theme Park Wiki API

In [3]:
import requests

url = "https://api.themeparks.wiki/v1/destinations"
response = requests.get(url)
data = response.json()

count = 1

for destination in data['destinations']:
    for park in destination['parks']:
        themepark = park['name']
        print(f"Theme Park {count}: {themepark}")
        count = count + 1

Theme Park 1: Chimelong Paradise
Theme Park 2: Chimelong Water Park
Theme Park 3: Chimelong Safari Park
Theme Park 4: Chimelong Birds Park
Theme Park 5: Walibi Holland
Theme Park 6: Parc Asterix
Theme Park 7: Plopsaland Belgium
Theme Park 8: California's Great America
Theme Park 9: Walibi Belgium
Theme Park 10: Silver Dollar City
Theme Park 11: Walibi Rhône-Alpes
Theme Park 12: Dollywood
Theme Park 13: Phantasialand
Theme Park 14: LEGOLAND Japan
Theme Park 15: Futuroscope
Theme Park 16: Chimelong Spaceship
Theme Park 17: Chimelong Ocean Kingdom
Theme Park 18: Hansa-Park
Theme Park 19: LEGOLAND Billund
Theme Park 20: Walt Disney Studios Park
Theme Park 21: Disneyland Park
Theme Park 22: Magic Kingdom Park
Theme Park 23: EPCOT
Theme Park 24: Disney's Hollywood Studios
Theme Park 25: Disney's Animal Kingdom Theme Park
Theme Park 26: Disney's Typhoon Lagoon Water Park
Theme Park 27: Disney's Blizzard Beach Water Park
Theme Park 28: Caribbean Bay
Theme Park 29: Everland
Theme Park 30: Eftel

# Part 1: Getting queue times live printed

There are over 100 theme parks in the entity. Since we are interested in Efteling queue data, the following code specifies its ID for futher API-use

In [4]:
entityID = None  #create entity variable for Efteling

for destination in data['destinations']:
    for park in destination['parks']:
        if park['name'] == 'Efteling':
            entityID = park['id']
            print(f"Stored entityID of Efteling: {entityID}")

Stored entityID of Efteling: 30713cf6-69a9-47c9-a505-52bb965f01be


In [5]:
#Specify the main endpote to the Efteling
url = f"https://api.themeparks.wiki/v1/entity/{entityID}/live"

response = requests.get(url)
response.raise_for_status()
efteling = response.json()

# Loop through all liveData items oer attraction
for item in efteling['liveData']:
    if item['entityType'] == 'ATTRACTION':
        name = item['name']
        status = item['status']
        standby_wait = item['queue']['STANDBY']['waitTime']
        single_rider_wait = item['queue'].get('SINGLE_RIDER', {}).get('waitTime')

        #To avoid data error when analysing, a closed or attraction in refurbishment get an N/A queue time
        #Additionally, a queue time of 0 was detected as None but for analysis this is modified to a 0
        if status in ['CLOSED', 'REFURBISHMENT']:
            standby_wait = 'N/A'
            if 'SINGLE_RIDER' in item['queue']:
                single_rider_wait = 'N/A'
        else:
            if standby_wait is None:
                standby_wait = 0
            if single_rider_wait is None:
                single_rider_wait = 0

        
        print(f"Attraction: {name}")
        print(f"  Status: {status}")
        print(f"  Wait Time: {standby_wait} minutes")
        
        # Only print single rider if the attraction HAS a single rider queue
        if 'SINGLE_RIDER' in item['queue']:
            print(f"  Single Rider Wait Time: {single_rider_wait} minutes")
        
        print("-" * 50)

Attraction: Droomvlucht
  Status: OPERATING
  Wait Time: 20 minutes
--------------------------------------------------
Attraction: Fata Morgana
  Status: OPERATING
  Wait Time: 5 minutes
--------------------------------------------------
Attraction: Villa Volta
  Status: OPERATING
  Wait Time: 5 minutes
--------------------------------------------------
Attraction: De Vliegende Hollander
  Status: OPERATING
  Wait Time: 5 minutes
  Single Rider Wait Time: 0 minutes
--------------------------------------------------
Attraction: Kleuterhof
  Status: OPERATING
  Wait Time: 0 minutes
--------------------------------------------------
Attraction: Nest!
  Status: OPERATING
  Wait Time: 0 minutes
--------------------------------------------------
Attraction: Archipel
  Status: OPERATING
  Wait Time: 0 minutes
--------------------------------------------------
Attraction: Sirocco
  Status: REFURBISHMENT
  Wait Time: N/A minutes
--------------------------------------------------
Attraction: Sto

Above code works and gives the live data. However, the results should be saved in a csv file. This is done below

# Part 2: Getting queue times live in a csv-file

In [8]:
import requests
import csv
from datetime import datetime

# Specify the main endpoint to the Efteling
url = f"https://api.themeparks.wiki/v1/entity/{entityID}/live"
response = requests.get(url)
response.raise_for_status()
efteling = response.json()

# Get current timestamp
timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

# Prepare data for CSV
csv_data = []

# Loop through all liveData items
for item in efteling['liveData']:
    if item['entityType'] == 'ATTRACTION':
        name = item['name']
        status = item['status']
        standby_wait = item['queue']['STANDBY']['waitTime']
        single_rider_wait = item['queue'].get('SINGLE_RIDER', {}).get('waitTime')
        
        # Handle standby queue
        if status in ['CLOSED', 'REFURBISHMENT']:
            standby_wait = 'N/A'
        else:
            if standby_wait is None:
                standby_wait = 0
        
        # Add standby queue row
        csv_data.append({
            'timestamp': timestamp,
            'attraction_name': name,
            'status': status,
            'queue_type': 'STANDBY',
            'wait_time': standby_wait
        })
        
        # Add single rider queue row if it exists
        if 'SINGLE_RIDER' in item['queue']:
            if status in ['CLOSED', 'REFURBISHMENT']:
                single_rider_wait = 'N/A'
            else:
                if single_rider_wait is None:
                    single_rider_wait = 0
            
            csv_data.append({
                'timestamp': timestamp,
                'attraction_name': name,
                'status': status,
                'queue_type': 'SINGLE_RIDER',
                'wait_time': single_rider_wait
            })

# Define CSV filename
csv_filename = 'efteling_queue_data.csv'

# Check if file exists to determine if we need to write headers
import os
file_exists = os.path.isfile(csv_filename)

# Write to CSV
with open(csv_filename, 'a', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['timestamp', 'attraction_name', 'status', 'queue_type', 'wait_time']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    
    # Only write header if file doesn't exist yet
    if not file_exists:
        writer.writeheader()
    
    # Write all rows
    writer.writerows(csv_data)

print(f"Data saved to {csv_filename} at {timestamp}")

Data saved to efteling_queue_data.csv at 2025-11-13 15:04:13


# Part 3: Automate queue data collection every 5 minutes for park openings

This is done in a seperate script